In [ ]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d subhajournal/android-malware-detection
unzip -qq android-malware-detection.zip

100%|██████████| 45.1M/45.1M [00:00<00:00, 77.5MB/s]


# Android Malware Classification

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from statistics import mode

In [8]:
df = pd.read_csv("Android_Malware.csv")
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (56,58,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,172.217.6.202-10.42.0.211-443-50004-6,10.42.0.211,50004,172.217.6.202,443.0,6.0,13/06/2017 11:52:39,37027,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
1,1,172.217.6.202-10.42.0.211-443-35455-6,10.42.0.211,35455,172.217.6.202,443.0,6.0,13/06/2017 11:52:39,36653,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
2,2,131.253.61.68-10.42.0.211-443-51775-6,10.42.0.211,51775,131.253.61.68,443.0,6.0,13/06/2017 11:52:42,534099,8,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
3,3,131.253.61.68-10.42.0.211-443-51775-6,10.42.0.211,51775,131.253.61.68,443.0,6.0,13/06/2017 11:52:43,9309,3,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
4,4,131.253.61.68-10.42.0.211-443-51776-6,10.42.0.211,51776,131.253.61.68,443.0,6.0,13/06/2017 11:52:42,19890496,8,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware


In [10]:
df.columns = df.columns.str.lstrip()

In [11]:
df.keys()

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [12]:
df.drop(["Unnamed: 0", "Flow ID", "Source IP", "Source Port", "Destination IP", "Timestamp"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355630 entries, 0 to 355629
Data columns (total 80 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             355630 non-null  float64
 1   Protocol                     355630 non-null  float64
 2   Flow Duration                355630 non-null  int64  
 3   Total Fwd Packets            355630 non-null  int64  
 4   Total Backward Packets       355630 non-null  int64  
 5   Total Length of Fwd Packets  355630 non-null  float64
 6   Total Length of Bwd Packets  355630 non-null  float64
 7   Fwd Packet Length Max        355630 non-null  float64
 8   Fwd Packet Length Min        355630 non-null  float64
 9   Fwd Packet Length Mean       355630 non-null  float64
 10  Fwd Packet Length Std        355630 non-null  float64
 11  Bwd Packet Length Max        355630 non-null  float64
 12  Bwd Packet Length Min        355630 non-null  float64
 13 

In [21]:
df["CWE Flag Count"].fillna(mode(df["CWE Flag Count"]), inplace=True)
df["Down/Up Ratio"].fillna(mode(df["Down/Up Ratio"]), inplace=True)
df["Fwd Avg Bytes/Bulk"].fillna(mode(df["Fwd Avg Bytes/Bulk"]), inplace=True)

In [24]:
for column in df.keys():
  if df[column].isna().sum() != 0:
    df[column].fillna(mode(df[column]), inplace=True)

In [25]:
df.keys()[:-1]

Index(['Destination Port', 'Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       

In [31]:
df["CWE Flag Count"].replace("SCAREWARE", 1, inplace=True)
df["CWE Flag Count"].astype("int64")

0         0
1         0
2         0
3         0
4         0
         ..
355625    0
355626    0
355627    0
355628    0
355629    0
Name: CWE Flag Count, Length: 355630, dtype: int64

In [33]:
df["CWE Flag Count"] = df["CWE Flag Count"].astype("int64")

In [35]:
df["Down/Up Ratio"].replace("BENIGN", 13, inplace=True)
df["Down/Up Ratio"] = df["Down/Up Ratio"].astype("int64")

In [38]:
df["Fwd Avg Bytes/Bulk"].replace("BENIGN", 1, inplace=True)
df["Fwd Avg Bytes/Bulk"] = df["Fwd Avg Bytes/Bulk"].astype("float64")

In [39]:
df.info(
  
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355630 entries, 0 to 355629
Data columns (total 80 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             355630 non-null  float64
 1   Protocol                     355630 non-null  float64
 2   Flow Duration                355630 non-null  int64  
 3   Total Fwd Packets            355630 non-null  int64  
 4   Total Backward Packets       355630 non-null  int64  
 5   Total Length of Fwd Packets  355630 non-null  float64
 6   Total Length of Bwd Packets  355630 non-null  float64
 7   Fwd Packet Length Max        355630 non-null  float64
 8   Fwd Packet Length Min        355630 non-null  float64
 9   Fwd Packet Length Mean       355630 non-null  float64
 10  Fwd Packet Length Std        355630 non-null  float64
 11  Bwd Packet Length Max        355630 non-null  float64
 12  Bwd Packet Length Min        355630 non-null  float64
 13 

In [40]:
hotEnc = OneHotEncoder()

In [44]:
df["Label"].unique()

array(['Android_Adware', 'Android_Scareware', 'Android_SMS_Malware',
       'Benign'], dtype=object)

In [56]:
le = LabelEncoder()
df["Label"] = le.fit_transform(df["Label"])
classes = le.classes_

In [57]:
classes

array(['Android_Adware', 'Android_SMS_Malware', 'Android_Scareware',
       'Benign'], dtype=object)

In [58]:
df.head()

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443.0,6.0,37027,1,1,0.0,0.0,0.0,0.0,0.000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,443.0,6.0,36653,1,1,0.0,0.0,0.0,0.0,0.000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,443.0,6.0,534099,8,12,1011.0,11924.0,581.0,0.0,126.375,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,443.0,6.0,9309,3,0,0.0,0.0,0.0,0.0,0.000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,443.0,6.0,19890496,8,6,430.0,5679.0,218.0,0.0,53.750,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [62]:
X = df.drop("Label", axis=1)
y = df["Label"]

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2)

In [63]:
len(X_train[0])

79

In [66]:
y_train[0]

2

In [67]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(128, input_shape=(len(X_train[0]), ), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(4, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [68]:
model.fit(X_train, y_train, epochs=120, validation_data=(X_test, y_test), batch_size=30)

Epoch 1/120
9484/9484 [==============================] - 33s 3ms/step - loss: 58870.1016 - accuracy: 0.3996 - val_loss: 14656.6396 - val_accuracy: 0.4166
Epoch 2/120
9484/9484 [==============================] - 29s 3ms/step - loss: 18779.5391 - accuracy: 0.4140 - val_loss: 1227.0774 - val_accuracy: 0.4166
Epoch 3/120
9484/9484 [==============================] - 31s 3ms/step - loss: 20629.3867 - accuracy: 0.4141 - val_loss: 1.3139 - val_accuracy: 0.4166
Epoch 4/120
9484/9484 [==============================] - 29s 3ms/step - loss: 1229.8271 - accuracy: 0.4141 - val_loss: 1.4891 - val_accuracy: 0.4166
Epoch 5/120
9484/9484 [==============================] - 31s 3ms/step - loss: 30812.7129 - accuracy: 0.4141 - val_loss: 1.4437 - val_accuracy: 0.4166
Epoch 6/120
9484/9484 [==============================] - 30s 3ms/step - loss: 10.7822 - accuracy: 0.4141 - val_loss: 1.4463 - val_accuracy: 0.4166
Epoch 7/120
9484/9484 [==============================] - 29s 3ms/step - loss: 2.5416 - accuracy: 

In [69]:
rclf = RandomForestClassifier()
rclf.fit(X_train, y_train)
rclf.score(X_test, y_test)

0.5263054298006354

In [75]:
rclf.predict([X_test[132]])

array([0])